In [42]:
path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo"

samco_path = r"\output\TextualPdf.pdf"
tata_path = r"\files\TataFactSheet2024.pdf"

In [43]:
import fitz 
import camelot
import pdfplumber
import collections

In [62]:

def annotate_pdf(input_path, output_path):
    # Open the PDF document
    pdf = fitz.open(input_path)
    
    # Iterate through each page
    for page_num in range(len(pdf)):
        page = pdf.load_page(page_num)
        # Extract text as a dictionary containing blocks, lines, and spans
        text_dict = page.get_text("dict")
        
        # Iterate through blocks
        for block in text_dict.get("blocks", []):
            # Check if the block contains lines
            if "lines" in block:
                # Iterate through lines
                for line in block["lines"]:
                    # Get the bounding box of the line
                    line_bbox = fitz.Rect(line["bbox"])
                    # Draw a rectangle around the line
                    page.draw_rect(line_bbox, color=(1, 0, 0), width=0.5)
    
    # Save the annotated PDF to the output path
    pdf.save(output_path)
    pdf.close()

# Example usage
input_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\TextualPage4.pdf" # Replace with your input PDF file path
output_pdf_path = r"C:\Users\Kaustubh.keny\OneDrive - Cogencis Information Services Ltd\Documents\mywork-repo\output\singular\sample.pdf"  # Replace with your desired output PDF file path
annotate_pdf(input_pdf_path, output_pdf_path)


In [44]:
def extract_pdf_blocks(input_pdf_path):
    """
    Open the PDF and extract all blocks of text, images, and other content.

    Args:
        input_pdf_path (str): Path to the input PDF.

    Returns:
        list: A list of pages, where each page is a dictionary containing blocks of content.
    """
    blocks_data = []
    
    input_doc = fitz.open(input_pdf_path)
    

    with pdfplumber.open(input_pdf_path) as pdf:
        for page_number, pdf_page in enumerate(pdf.pages):
            # Extract blocks of content using PyMuPDF
            doc_page = input_doc[page_number]
            blocks = doc_page.get_text("dict")["blocks"]
            
            #get the color and size for each block then aggregate
            page_color = []
            page_size = []
            for block in blocks:
                if 'lines' not in block:
                    continue
                for lines in block['lines']:
                    for span in lines['spans']:
                        color = span['color']
                        size = span['size']
                        page_color.append(color)
                        page_size.append(size)
                        
            page_color = collections.Counter(page_color)
            page_size = collections.Counter(page_size)
            # Append the blocks along with table bounding boxes from pdfplumber
            blocks_data.append({
                "blocks": blocks,
                #"table_bboxes": [table.bbox for table in pdf_page.find_tables()],
                #"page_rect": doc_page.rect,
                "page_colors":dict(page_color), #color for each line
                "page_sizes": dict(page_size),
                #"total_pages": input_doc.page_count
            })

    input_doc.close()
    return blocks_data

In [45]:
pdf_data = extract_pdf_blocks(path + samco_path)

In [ ]:
pdf_colors = []

for page in pdf_data:
    temp = list(page['page_colors'].keys())
    pdf_colors.extend(temp)

pdf_colors = list(set(pdf_colors))
pdf_colors

In [ ]:
def extract_rgb(color_int):
    """
    Extract RGB components from an ARGB integer.
    Ignores the alpha component.
    """
    red = (color_int >> 16) & 0xFF
    green = (color_int >> 8) & 0xFF
    blue = color_int & 0xFF
    return (red, green, blue)0

def closest_color(rgb_value):
    """
    Finds the closest color name for a given RGB value.
    """
    colors = {
        'Black': (0, 0, 0),
        'White': (255, 255, 255),
        'Red': (255, 0, 0),
        'Lime': (0, 255, 0),
        'Blue': (0, 0, 255),
        'Yellow': (255, 255, 0),
        'Cyan': (0, 255, 255),
        'Magenta': (255, 0, 255),
        'Silver': (192, 192, 192),
        'Gray': (128, 128, 128),
        'Maroon': (128, 0, 0),
        'Olive': (128, 128, 0),
        'Green': (0, 128, 0),
        'Purple': (128, 0, 128),
        'Teal': (0, 128, 128),
        'Navy': (0, 0, 128)
    }

    min_distance = float('inf')
    closest_color_name = None
    for name, (r_c, g_c, b_c) in colors.items():
        rd = (r_c - rgb_value[0]) ** 2
        gd = (g_c - rgb_value[1]) ** 2
        bd = (b_c - rgb_value[2]) ** 2
        distance = rd + gd + bd
        if distance < min_distance:
            min_distance = distance
            closest_color_name = name

    return closest_color_name

for color_int in pdf_colors:
    rgb = extract_rgb(color_int)
    color_name = closest_color(rgb)
    print("RGB:", rgb, "Closest color name:", color_name)


In [55]:
from PIL import Image, ImageDraw

def create_color_image(colors, block_size=50):
    """
    Creates an image with given colors shown as blocks.

    Args:
    colors (list of tuples): List of (A, R, G, B) tuples.
    block_size (int): Size of each color block in pixels.

    Returns:
    Image: The created PIL Image object.
    """
    # Determine the image size
    num_colors = len(colors)
    width = block_size
    height = block_size * num_colors
    
    # Create a new image with white background
    image = Image.new("RGBA", (width, height), (255, 255, 255, 255))
    draw = ImageDraw.Draw(image)
    
    # Draw each color as a block
    for i, color in enumerate(colors):
        # Calculate the position of the block
        top = i * block_size
        bottom = top + block_size
        # Draw the rectangle
        draw.rectangle([0, top, width, bottom], fill=color)
    
    return image


# Create the image
color_image = create_color_image(pdf_colors)

# Display the image
color_image.show()


In [59]:
def closest_color(rgb_value):
    # A dictionary of some basic color names and their RGB values
    colors = {
        'Black': (0, 0, 0),
        'White': (255, 255, 255),
        'Red': (255, 0, 0),
        'Lime': (0, 255, 0),
        'Blue': (0, 0, 255),
        'Yellow': (255, 255, 0),
        'Cyan': (0, 255, 255),
        'Magenta': (255, 0, 255),
        'Silver': (192, 192, 192),
        'Gray': (128, 128, 128),
        'Maroon': (128, 0, 0),
        'Olive': (128, 128, 0),
        'Green': (0, 128, 0),
        'Purple': (128, 0, 128),
        'Teal': (0, 128, 128),
        'Navy': (0, 0, 128)
    }
    min_distance = float('inf')
    closest_color = None

    for color, value in colors.items():
        distance = sum((s - q) ** 2 for s, q in zip(rgb_value, value))
        if distance < min_distance:
            min_distance = distance
            closest_color = color

    return closest_color



for color in pdf_colors:
    print(closest_color(color))

TypeError: 'int' object is not iterable